In [1]:
%load_ext autoreload
%autoreload

In [2]:
import syft as sy
import numpy as np
from syft.core.adp.entity import Entity
sy.logger.remove()

In [3]:
domain1 = sy.login(email="info@openmined.org", password="changethis", port=8081)
domain2 = sy.login(email="info@openmined.org", password="changethis", port=8082)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into canada... done!

Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into italy... done!


In [42]:
domain1.users

,id,email,name,budget,verify_key,role,added_by,website,institution,daa_pdf,created_at,budget_spent
0,1,info@openmined.org,Jane Doe,5.55,e16384f64d5aaa351235a9d75531ea9f500e897a989592...,Owner,None,None,None,NaN,2022-03-03 11:03:00.567137,5.550000
1,2,sheldon@caltech.edu,Sheldon Cooper,200.00,ab6992db0aa875d47fd2b377e92feae135f56a4a3b7a1c...,Data Scientist,Jane Doe,,,1.0,2022-03-09 16:28:42.447273,98.551903
2,3,cd0613a2-fc09-4055-8381-504cfc1a54a5@caltech.edu,Sheldon Cooper,10.00,eb30bf6d31c478aebd9e7021333de793544ec00ad7e4fe...,Data Scientist,Jane Doe,,,2.0,2022-03-09 16:57:26.496910,10.000000
3,4,41aa6ef4-0aa5-49bb-8a56-3e8124f80a82@caltech.edu,Sheldon Cooper,210.00,27d6f45bb0c1d0228e9e9d8a8d87bdec8858970c846eff...,Data Scientist,Jane Doe,,,3.0,2022-03-09 16:57:43.625608,116.965649
4,5,16b23028-d9db-4f77-bad2-23c1cbf02966@caltech.edu,Sheldon Cooper,210.00,f9a5fd5e8cfdd3cedf7116cf445a0c184f2607e6effc59...,Data Scientist,Jane Doe,,,4.0,2022-03-09 17:02:28.740731,51.567847
5,6,b4b72d36-ccda-4ec4-9cbe-8e4b9cd722bc@caltech.edu,Sheldon Cooper,210.00,720c551ac7955ec1295a758aa6e87ac86cf9d935ef0859...,Data Scientist,Jane Doe,,,5.0,2022-03-09 17:05:41.568509,49.330925


In [44]:
domain2.users

,id,email,name,budget,verify_key,role,added_by,website,institution,daa_pdf,created_at,budget_spent
0,1,info@openmined.org,Jane Doe,5.55,16a83ec72ae1691ae69ed825aea0c88a927fbf31d74dd7...,Owner,None,None,None,NaN,2022-03-03 11:03:27.098890,5.550000
1,2,sheldon@caltech.edu,Sheldon Cooper,200.00,65d8967d85aac13d5416f94d0894f0304834818c54ac8e...,Data Scientist,Jane Doe,,,1.0,2022-03-09 16:34:59.356103,98.551903
2,3,cd0613a2-fc09-4055-8381-504cfc1a54a5@caltech.edu,Sheldon Cooper,10.00,204da300dfc5f9d6954548f6017c6b7f09b4abde34bfbf...,Data Scientist,Jane Doe,,,2.0,2022-03-09 16:57:28.897189,10.000000
3,4,41aa6ef4-0aa5-49bb-8a56-3e8124f80a82@caltech.edu,Sheldon Cooper,210.00,65f37a0926e3f0800f6f03a722e98b7b51fa5b0783960c...,Data Scientist,Jane Doe,,,3.0,2022-03-09 16:57:46.034250,116.965649
4,5,16b23028-d9db-4f77-bad2-23c1cbf02966@caltech.edu,Sheldon Cooper,210.00,2a4b7933066ca6ca0066d061620ff2e2c3d571d8050bbc...,Data Scientist,Jane Doe,,,4.0,2022-03-09 17:02:31.270027,51.567847
5,6,b4b72d36-ccda-4ec4-9cbe-8e4b9cd722bc@caltech.edu,Sheldon Cooper,210.00,f85244c0c58c866874cd58e9ec899eeec7a337a6440711...,Data Scientist,Jane Doe,,,5.0,2022-03-09 17:05:53.901936,49.330925


In [45]:

for domain in [domain1,domain2]:
    if domain.users.pandas().shape[0] == 1:
        domain.users.create(
            **{
                "name": "Sheldon Cooper",
                "email": "sheldon@caltech.edu",
                "password": "bazinga",
                "budget":200
            }
        )
        
        domain.users

In [46]:
data = np.random.randint(1, 100, size=(5, 5), dtype=np.int32)
ishan = Entity(name="Ishan")

tensor = sy.Tensor(data).private(min_val=0, max_val=100, entities=ishan)

In [7]:
data

array([[36, 58, 57, 58, 27],
       [40, 29, 19, 13, 75],
       [89, 12, 10, 97, 16],
       [69, 93, 48, 24,  7],
       [96, 99, 41, 45,  4]], dtype=int32)

In [47]:
domain1.load_dataset(
    assets={"data": tensor},
    name="Sample data",
    description="Sample data to test privacy budget is being updated"
)
domain2.load_dataset(
    assets={"data": tensor},
    name="Sample data",
    description="Sample data to test privacy budget is being updated"
)


Loading dataset... uploading... SUCCESS!                        

Run <your client variable>.datasets to see your new dataset loaded into your machine!
Loading dataset... uploading... SUCCESS!                        

Run <your client variable>.datasets to see your new dataset loaded into your machine!


In [48]:
domain1.datasets

Idx,Name,Description,Assets,Id
[0],Sample data,Sample data to test privacy budget is being updated,"[""data""] -> Tensor",e5616a3a-f58e-48de-9500-ebcc0a3f4d4e
[1],Sample data,Sample data to test privacy budget is being updated,"[""data""] -> Tensor",b9fe3680-c931-440e-8792-7f61823fe1fa
[2],Sample data,Sample data to test privacy budget is being updated,"[""data""] -> Tensor",4f4fe946-b072-40a9-bd38-7f72c9c86d00
[3],Canada Trade Data - First few rows,A collection of reports from Canada's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Canada Trade""] -> Tensor",f4572bf4-e693-4b7f-97ad-4814a1edeff7
[4],Canada Trade Data - First few rows,A collection of reports from Canada's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Canada Trade""] -> Tensor",cf35f228-5db8-4d40-9beb-4ec28b7f2e0f
[5],Canada Trade Data - First few rows,A collection of reports from Canada's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Canada Trade""] -> Tensor",d854aa89-93d5-4bce-ba48-33f016a8d300
[6],Canada Trade Data - First few rows,A collection of reports from Canada's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Canada Trade""] -> Tensor",68f847ab-180c-4b5e-8992-ceb932db7317
[7],Canada Trade Data - First few rows,A collection of reports from Canada's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Canada Trade""] -> Tensor",7318a1d9-456e-404e-80ef-e182569d399e
[8],Sample data,Sample data to test privacy budget is being updated,"[""data""] -> Tensor",a5554db9-3dcc-4212-9173-3cfd9ea8366c


## Log in as Data Scientist

In [4]:
ds1 = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)
ds2 = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8082)

Connecting to None... done! 	 Logging into canada... done!
Connecting to None... done! 	 Logging into italy... done!


In [50]:
ds1.datasets

Idx,Name,Description,Assets,Id
[0],Sample data,Sample data to test privacy budget is being updated,"[""data""] -> Tensor",e5616a3a-f58e-48de-9500-ebcc0a3f4d4e
[1],Sample data,Sample data to test privacy budget is being updated,"[""data""] -> Tensor",b9fe3680-c931-440e-8792-7f61823fe1fa
[2],Sample data,Sample data to test privacy budget is being updated,"[""data""] -> Tensor",4f4fe946-b072-40a9-bd38-7f72c9c86d00
[3],Canada Trade Data - First few rows,A collection of reports from Canada's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Canada Trade""] -> Tensor",f4572bf4-e693-4b7f-97ad-4814a1edeff7
[4],Canada Trade Data - First few rows,A collection of reports from Canada's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Canada Trade""] -> Tensor",cf35f228-5db8-4d40-9beb-4ec28b7f2e0f
[5],Canada Trade Data - First few rows,A collection of reports from Canada's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Canada Trade""] -> Tensor",d854aa89-93d5-4bce-ba48-33f016a8d300
[6],Canada Trade Data - First few rows,A collection of reports from Canada's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Canada Trade""] -> Tensor",68f847ab-180c-4b5e-8992-ceb932db7317
[7],Canada Trade Data - First few rows,A collection of reports from Canada's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Canada Trade""] -> Tensor",7318a1d9-456e-404e-80ef-e182569d399e
[8],Sample data,Sample data to test privacy budget is being updated,"[""data""] -> Tensor",a5554db9-3dcc-4212-9173-3cfd9ea8366c


In [56]:
ds1.request_budget(eps=200,reason="I really want it.")
ds2.request_budget(eps=200,reason="I really want it.")

Requested 200 epsilon of budget. Call .privacy_budget to see if your budget has arrived!
Requested 200 epsilon of budget. Call .privacy_budget to see if your budget has arrived!


In [60]:
domain1.requests[-1].accept()
domain2.requests[-1].accept()

In [7]:
ds1.privacy_budget

298.55190303438684

In [8]:
ptr1 = ds1.datasets[-1]["data"]
ptr2 = ds2.datasets[-1]["data"]

In [63]:
print(ptr1.client)
print(ptr2.client)

<DomainClient - canada: <UID: b0ddc973a42e4f24bfd8db02a13f91dd>>
<DomainClient - italy: <UID: 3eba7458a9ac4907ad30e5231857b2d1>>


In [31]:
result = ptr1 + ptr2

In [65]:
type(result)

syft.core.tensor.smpc.mpc_tensor.MPCTensor

In [32]:
published_result = result.publish(sigma=100)

In [33]:
ds1.privacy_budget

34.40456180157662

In [19]:
published_result.child[1].exists

True

In [34]:
published_result.get()

You do not have sufficient privacy budget. Kindly request for additional privacy budget. 

Before clip [[ 113  215 -420  -80  183]
 [-215 -241  -72 -259   -9]
 [  29   58  270 -273    4]
 [ -85  135  111  107   50]
 [  59  -27  -99   58  118]]
min_vals [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
max_vals [[200 200 200 200 200]
 [200 200 200 200 200]
 [200 200 200 200 200]
 [200 200 200 200 200]
 [200 200 200 200 200]]


array([[113, 200,   0,   0, 183],
       [  0,   0,   0,   0,   0],
       [ 29,  58, 200,   0,   4],
       [  0, 135, 111, 107,  50],
       [ 59,   0,   0,  58, 118]], dtype=int32)

In [24]:
ds2.privacy_budget

200.0